In [10]:
# import package
import pandas as pd
import mysql.connector
from datetime import datetime

In [11]:
df = pd.read_excel("traffic_stops.xlsx")

In [12]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [13]:
# data cleaning
df.dropna(axis=1,how='all',inplace=True)


In [14]:
# Fill NaNs with suitable values
df.fillna({
    'driver_age':df['driver_age'].median(),
    'search_type':'None',
    'stop_duration':'Unknown',
    'violation':'Unknown',
    'stop_outcome':'Unknown'
},inplace=True)

In [15]:
# convert date & time to timestamp
df['timestamp'] = pd.to_datetime(
    df['stop_date'].astype(str) + ' ' + df['stop_time'].astype(str)
)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stop_date           65538 non-null  datetime64[ns]
 1   stop_time           65538 non-null  object        
 2   country_name        65538 non-null  object        
 3   driver_gender       65538 non-null  object        
 4   driver_age_raw      65538 non-null  int64         
 5   driver_age          65538 non-null  int64         
 6   driver_race         65538 non-null  object        
 7   violation_raw       65538 non-null  object        
 8   violation           65538 non-null  object        
 9   search_conducted    65538 non-null  bool          
 10  search_type         65538 non-null  object        
 11  stop_outcome        65538 non-null  object        
 12  is_arrested         65538 non-null  bool          
 13  stop_duration       65538 non-null  object    

In [17]:
! pip install pymysql

In [18]:
import pymysql

In [19]:
# connect to my sql

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Surya@1516"
    
)

In [20]:
# cursor connection
cursor=connection.cursor()


In [21]:
cursor.execute("create database securecheck")

ProgrammingError: (1007, "Can't create database 'securecheck'; database exists")

In [22]:
cursor.execute("USE securecheck;")

0

In [23]:
cursor.execute(""" CREATE TABLE IF NOT EXISTS police_logs(
id INT AUTO_INCREMENT PRIMARY KEY,
stop_date DATE,
stop_time TIME,
country_name VARCHAR(100),
driver_gender VARCHAR(10),
driver_age INT,
driver_race VARCHAR(50),
violation VARCHAR(100),
search_conducted BOOLEAN,
search_type VARCHAR(100),
stop_outcome VARCHAR(100),
drugs_related_stop BOOLEAN,
stop_duration VARCHAR(50),
vehicle_number VARCHAR(50),
timestamp DATETIME

)

""")

0

In [25]:
# data cleaning
df.dropna(axis=1,how='all',inplace=True)
df.fillna({
    'driver_age':df['driver_age'].median(),
    'search_type':'None',
    'stop_duration':'Unknown',
    'violation':'Unknown',
    'stop_outcome':'Unknown'
},inplace=True)
# create timestamp column
df['timestamp'] = pd.to_datetime(
    df['stop_date'].astype(str) + ' ' + df['stop_time'].astype(str)
)


In [26]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,None,Warning,True,0-15 Min,True,KA24UZ8488,2020-02-15 12:16:00


In [27]:
# convert stop date to correct format(YYYY-MM-DD)
df['stop_date']=pd.to_datetime(df['stop_date'],errors='coerce').dt.strftime('%Y-%m-%d')
df['stop_time']=pd.to_datetime(df['stop_time'],errors='coerce').dt.strftime('%H:%M:%S')


In [28]:
# Replace NaN with none
df=df.where(pd.notnull(df),None)

In [29]:
# prepare insert query
insert_query="""
INSERT INTO police_logs(	
     stop_date,stop_time,country_name,driver_gender,driver_age,
     driver_race,violation,search_conducted,search_type,stop_outcome,
     drugs_related_stop,stop_duration,vehicle_number,timestamp
     )VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

In [31]:
# Iterate DataFrame and insert rows
for _,row in df.iterrows():
    cursor.execute(insert_query,(
     row['stop_date'],
     row['stop_time'],
     row.get('country_name'),
     row.get('driver_gender'),
     int(row.get('driver_age'))if row.get('driver_age') is not None else None,
     row.get('driver_race'),
     row.get('violation'),
     bool(row.get('search_conducted'))if row.get('search_conducted')is not None else None,
     row.get('search_type'),
     row.get('stop_outcome'),
     bool(row.get('drugs_related_stop'))if row.get('drugs_related_stop')is not None else None,
     row.get('stop_duration'),
     row.get('vehicle_number',f'VEH{row.name}'),
     row['timestamp']
    ))

In [32]:
# Commit and Close connection
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully into single table! ")

Data inserted successfully into single table! 
